Adım 1: Veriyi Okuma

**Adım 3: Yeni Veriyi Dosyaya Kaydetme**

**Adım 4: Yeni Veriyi Kontrol Edin**

**Adım 5: Artırılmış Veri ile Model Eğitimi**

Adım 5.1: Yeni Veriyi Eğitim ve Test Setlerine Bölme

Adım 5.2: Veriyi Tokenize Etme

Adım 5.3: Tensor Formatına Dönüştürme

Adım 5.4: Model Eğitimi

Adım 5.5: Performans Değerlendirmesi

**Adım 6: Model Performansının Detaylı Değerlendirilmesi**

Adım 6.1: Karmaşıklık Matrisi ve Sınıflandırma Raporu

Adım 6.2: Karmaşıklık Matrisi Görselleştirme

Adım 6.3: Precision, Recall ve F1-Score Görselleştirme

Adım 6.4: Tahmin Sonuçlarının Örnek Analizi

**Adım 7: Modelin Gerçek Hayatta Kullanımı**

Adım 7.1: Gerçek Log Verileri ile Tahmin

Adım 7.2: Tahmin Sonuçlarının Analizi

Adım 7.3: Tahmin Dağılımının Görselleştirilmesi

Adım 7.4: Yeni Veriler için Anomali Tespiti

Adım 7: Model Performansının Sonuçlarını Analiz Etme

Adım 7.1: Model Performansının Yorumlanması

In [ ]:

import re
import pandas as pd

# Log dosyasını yükleme ve ayrıştırma
def parse_log(log):
    log_pattern = r'(?P<ip>\d+\.\d+\.\d+\.\d+) - - \[(?P<timestamp>.*?)\] "(?P<method>.*?) (?P<url>.*?) (?P<protocol>.*?)" (?P<status>\d+) (?P<size>\d+|-)'
    match = re.match(log_pattern, log)
    return match.groupdict() if match else None

# Log dosyasını liste olarak yükleme
logs = []  # Log dosyanız burada listelenmeli

# Logları DataFrame'e dönüştürme
parsed_logs = [parse_log(log) for log in logs if parse_log(log)]
logs_df = pd.DataFrame(parsed_logs)

# Etiketleri ekleme
logs_df['label'] = logs_df['status'].apply(lambda x: 1 if x.startswith('4') or x.startswith('5') else 0)
    

In [ ]:

import random

# Sentetik veri üretim fonksiyonu
def generate_synthetic_data(df, target_size=10000):
    # Zaman damgasını datetime formatına dönüştürme
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        df = df.dropna(subset=['timestamp'])

    synthetic_data = df.copy()
    rows = []

    # Sentetik veri üretimi
    while len(synthetic_data) + len(rows) < target_size:
        for _, row in df.iterrows():
            manipulated_row = row.copy()
            manipulated_row['ip'] = f"{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}"
            manipulated_row['timestamp'] = manipulated_row['timestamp'] + pd.Timedelta(seconds=random.randint(1, 3600))
            manipulated_row['method'] = random.choice(["GET", "POST", "HEAD", "OPTIONS", "DELETE"])
            manipulated_row['status'] = random.choice(["200", "404", "500", "403", "302"])
            manipulated_row['url'] = manipulated_row['url'] + f"?query={random.randint(1, 100)}"
            rows.append(manipulated_row)

            if len(synthetic_data) + len(rows) >= target_size:
                break

    # Performans için tüm satırları birleştir
    synthetic_data = pd.concat([synthetic_data, pd.DataFrame(rows)], ignore_index=True)

    print(f"Sentetik veri üretildi: {len(synthetic_data)} kayıt.")
    return synthetic_data
    

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE

# Sınıf dengesini sağlama fonksiyonu
def balance_classes_with_smote(df):
    if 'url' not in df.columns or 'label' not in df.columns:
        raise ValueError("DataFrame'de 'url' veya 'label' sütunu eksik!")

    # TF-IDF ile sayısallaştırma
    vectorizer = TfidfVectorizer(max_features=1000)
    X_numeric = vectorizer.fit_transform(df['url']).toarray()
    y = df['label']

    # SMOTE ile dengeleme
    smote = SMOTE(random_state=42)
    X_balanced, y_balanced = smote.fit_resample(X_numeric, y)

    print(f"Sınıf dengesi sağlandı: {len(X_balanced)} kayıt.")
    return X_balanced, y_balanced
    

In [ ]:

# Tüm süreci birleştirme
def process_data(df, target_size=10000):
    # Sentetik veri üretimi
    synthetic_data = generate_synthetic_data(df, target_size)

    # Sınıf dengesini sağlama
    X_balanced, y_balanced = balance_classes_with_smote(synthetic_data)

    return X_balanced, y_balanced

# Kullanım
X_balanced, y_balanced = process_data(logs_df, target_size=10000)
print(f"Sonuç: Dengeli veri boyutu: {len(X_balanced)}")
    